# Trying out MPE2 simple (base model)

First, let's start with the general framework for running the experiment. The simple environment has only one agent and one landmark. They have random color. First, let's try a random action.

In [ ]:

from mpe2 import simple_v3
env = simple_v3.env(max_cycles=100, render_mode='human')

env.reset() # Do seed=42 for reproducibility
for agent in env.agent_iter():
    observation, reward, termination, truncation, info = env.last()

    if termination or truncation:
        action = None
    else:
        action = env.action_space(agent).sample() # this is where you would insert your policy

    env.step(action)
env.close()

Now, let's try with our own environment particle_v1:

In [ ]:
import particle_v1
env = particle_v1.env(num_agents=10, num_food_sources=1, flow = 'none', max_cycles=25, render_mode='human')

env.reset() # Do seed=42 for reproducibility
for agent in env.agent_iter():
    observation, reward, termination, truncation, info = env.last()

    if termination or truncation:
        action = None
    else:
        action = env.action_space(agent).sample() # this is where you would insert your policy

    env.step(action)
env.close()